# Sprint 4 — Webinar 13 (Práctica)

**Modalidad:** Coding Together + Breakrooms  
**Tema:** Modelar escenarios de funnels y estimar impacto de negocio (SQLite)  
**Versión:** 2025-11-09T10:00:00

> Esta sesión práctica parte de lo aprendido en **S4W11 (Teórico)** y **S4W12 (Práctico)**. Ahora vamos a:
> - Simular mejoras en un funnel de conversión.
> - Traducir mejoras de conversión en impacto de negocio (ingresos).
> - Comparar escenarios entre segmentos y variantes A/B.
> - Preparar visualizaciones simples para explicarlo a *stakeholders*.


### Agenda – Webinar 13 (Práctico)

| Hora | Actividad | Descripción |
|------|------------|-------------|
| 00:00 – 00:10 | **Introducción y repaso teórico** | Breve repaso del embudo de conversión, eventos principales y estructura de datos utilizada. |
| 00:10 – 00:25 | **Ejercicio 0 (Breakroom)** | Actividad inicial en grupos pequeños: identificar usuarios únicos y su primera interacción. |
| 00:25 – 00:50 | **Construcción de CTEs por etapa** | Creación colaborativa de CTEs (`first_visit`, `select_item`, `add_to_cart`, `purchase`) aplicando filtros y deduplicación. |
| 00:50 – 01:10 | **Unión del embudo completo** | Combinar CTEs en un único flujo de usuarios que pasaron por cada etapa; análisis de pérdidas entre etapas. |
| 01:10 – 01:20 | **Análisis de métricas** | Cálculo de tasas de conversión, abandono y eficiencia del embudo usando consultas SQL. |
| 01:20 – 01:25 | **Mini–Kahoot / Evaluación rápida** | Preguntas cortas para verificar comprensión (JOINs, CTEs, filtros). |
| 01:25 – 01:30 | **Cierre y próximos pasos** | Conclusiones, recomendaciones de práctica y preparación para el siguiente webinar. |


## 0. Objetivos de aprendizaje

Al final de este webinar deberías ser capaz de:

1. **Explicar qué es un A/B test** en el contexto de funnels y por qué se usa.  
2. **Distinguir** entre cambios orgánicos en la conversión y el efecto real de un tratamiento.  
3. **Simular mejoras** en una etapa del funnel usando consultas SQL.  
4. **Asignar valor/revenue** a las conversiones y estimar el impacto total en el negocio.  
5. **Comparar escenarios por segmento** (ej. web vs mobile) para ver quién aporta más valor.  
6. **Preparar una visualización simple** (Actual vs Simulado) en Google Sheets y escribir conclusiones breves.


## 0.1. Cómo usar este notebook con SQLite Online

En este webinar **no conectaremos Python a una base de datos**: usaremos la herramienta web `SQLiteOnline`.

Pasos recomendados:

1. Abre la página de tu preferencia para trabajar con **SQLite online**.  
2. Elige el motor **SQLite**.  
3. Copia y pega los bloques de código SQL marcados como:

   ```sql
   -- ejemplo
   SELECT 1;
   ```

4. Ejecuta cada bloque por separado:
   - Primero el bloque de **DDL + PRAGMA** (creación de tablas).
   - Luego el bloque de **INSERTs** (carga de datos).
   - Después, las **consultas de cada ejercicio**.

Puedes ir marcando en el notebook qué ejercicios ya completaste.


## 1. Recordatorio teórico — Funnels y A/B testing

### 1.1 Funnels de conversión

Un **funnel** (embudo) representa las etapas que recorre un usuario hasta completar una acción de negocio, por ejemplo:

1. `view_product` — visita ficha de producto  
2. `add_to_cart` — agrega al carrito  
3. `begin_checkout` — inicia checkout  
4. `purchase` — compra completada  

En cada paso, algunos usuarios **abandonan** y otros avanzan. Por eso hablamos de:

- **Tasa de paso** entre etapas, ej.:  
  - `add_to_cart_rate = usuarios_con_add_to_cart / usuarios_con_view_product`
- **Tasa de conversión final**, ej.:  
  - `purchase_rate = usuarios_con_purchase / usuarios_con_view_product`

### 1.2 A/B testing en funnels

Un **A/B test** es un experimento controlado donde se comparan dos variantes:

- **A (control):** versión actual (ej. diseño original del checkout).  
- **B (tratamiento):** nueva versión (ej. botón más destacado, menos pasos, etc.).

Objetivo: responder si la variante **B mejora la conversión** *más allá* de la variabilidad natural.

En este webinar:

- Trabajaremos con una tabla donde cada sesión tiene:
  - `variant` (A o B),
  - flags de si pasó cada etapa,
  - `revenue` asociado si compró.
- Simularemos mejoras *hipotéticas* en una etapa (ej. +5 p.p. en `add_to_cart -> begin_checkout` para B).  

> **Importante:** aquí haremos solo **simulación analítica**. La validación estadística formal (test de hipótesis, significancia, etc.) vendrá en un módulo posterior.


### 1.3 Del funnel al impacto de negocio

Para hablar con negocio necesitamos traducir conversiones a **dinero**:

- Asignamos un **valor por conversión** (ej. ticket promedio por compra).  
- Calculamos:

  - `revenue_total = SUM(revenue)`  
  - `revenue_por_usuario`, `revenue_por_sesión`, etc.

Cuando simulamos una mejora de funnel:

1. Estimamos cuántos usuarios adicionales llegarían a `purchase`.  
2. Multiplicamos por el **ticket promedio** o `revenue` medio por compra.  
3. Comparamos **Escenario Actual vs Escenario Simulado**.

### 1.4 Segmentos y heterogeneidad

No todos los usuarios son iguales:

- Segmentos típicos: `web vs mobile`, `new vs returning`, `free vs paid`, etc.
- Una misma mejora de funnel puede tener impacto distinto por segmento.

Por eso nos interesa:

- Comparar conversiones y revenue por `segment`.  
- Identificar **qué segmento aporta más** al uplift total si aplicamos la mejora.

### 1.5 Visualización para stakeholders

Para explicar resultados a personas no técnicas:

- Usamos gráficos sencillos (barras, funnels, columnas agrupadas).
- Mostramos:

  - **Actual vs Simulado** por etapa.  
  - **Actual vs Simulado** en revenue total o por segmento.

- Añadimos 3–5 frases claras:
  - Qué cambió en la tasa de conversión.
  - Cuánto impacto económico estimado genera.
  - Qué segmento se beneficia más.

Con esto en mente, pasemos a crear nuestro dataset de ejemplo.


In [ ]:
-- ==============================================
-- DDL S4W13 para SQLiteOnline (sin esquemas)
-- Pega y ejecuta este bloque primero.
-- ==============================================

-- 1) Asegura FK activas en SQLite
PRAGMA foreign_keys = ON;

-- 2) Limpieza idempotente (por si vuelves a correr el script)
DROP TABLE IF EXISTS funnel_sessions;
DROP TABLE IF EXISTS users;

-- 3) Tabla de usuarios (segmentos)
CREATE TABLE users (
  user_id   INTEGER PRIMARY KEY,
  segment   TEXT NOT NULL,       -- 'web' o 'mobile'
  country   TEXT NOT NULL        -- por simplicidad, pocos países
);

-- 4) Tabla de sesiones de funnel
-- Cada fila ~ 1 sesión de usuario en el sitio
CREATE TABLE funnel_sessions (
  session_id        INTEGER PRIMARY KEY,
  user_id           INTEGER NOT NULL,
  variant           TEXT NOT NULL,    -- 'A' (control) o 'B' (tratamiento)
  viewed_product    INTEGER NOT NULL, -- 1 si vio ficha de producto
  added_to_cart     INTEGER NOT NULL, -- 1 si agregó al carrito
  begin_checkout    INTEGER NOT NULL, -- 1 si inició checkout
  purchased         INTEGER NOT NULL, -- 1 si completó compra
  revenue           REAL NOT NULL,    -- importe de la compra (0 si no compró)
  session_date      TEXT NOT NULL,    -- ISO-8601: '2025-10-01'
  FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);

-- Nota de modelado:
-- En un sistema real tendríamos tabla de eventos a nivel click/etapa.
-- Aquí usamos flags 0/1 por etapa para simplificar el análisis en clase.


In [ ]:
-- ==============================================
-- Seed de usuarios (segmentos)
-- Ejecuta este bloque después del DDL.
-- ==============================================

INSERT INTO users (user_id, segment, country) VALUES
  (1,  'web',    'CO'),
  (2,  'web',    'CO'),
  (3,  'web',    'MX'),
  (4,  'web',    'MX'),
  (5,  'web',    'AR'),
  (6,  'web',    'CO'),
  (7,  'mobile', 'CO'),
  (8,  'mobile', 'CO'),
  (9,  'mobile', 'MX'),
  (10, 'mobile', 'MX'),
  (11, 'mobile', 'AR'),
  (12, 'mobile', 'CO'),
  (13, 'web',    'CO'),
  (14, 'web',    'CO'),
  (15, 'mobile', 'CO'),
  (16, 'mobile', 'CO'),
  (17, 'web',    'MX'),
  (18, 'mobile', 'MX'),
  (19, 'web',    'AR'),
  (20, 'mobile', 'AR');


In [ ]:
-- ==============================================
-- Seed de sesiones de funnel
-- Supone un mes de observación con A/B test en marcha.
-- ==============================================

INSERT INTO funnel_sessions (
  session_id, user_id, variant,
  viewed_product, added_to_cart, begin_checkout, purchased,
  revenue, session_date
) VALUES
  -- Variante A (control) - segmento web
  ( 1,  1, 'A', 1,1,1,1, 120.0, '2025-10-01'),
  ( 2,  1, 'A', 1,1,1,0,   0.0, '2025-10-05'),
  ( 3,  2, 'A', 1,1,0,0,   0.0, '2025-10-03'),
  ( 4,  2, 'A', 1,0,0,0,   0.0, '2025-10-07'),
  ( 5,  3, 'A', 1,1,1,1,   80.0, '2025-10-02'),
  ( 6,  3, 'A', 1,1,1,0,   0.0, '2025-10-08'),
  ( 7,  4, 'A', 1,1,0,0,   0.0, '2025-10-04'),
  ( 8,  5, 'A', 1,0,0,0,   0.0, '2025-10-06'),
  ( 9,  6, 'A', 1,1,1,1, 150.0, '2025-10-09'),
  (10,  6, 'A', 1,1,1,1, 130.0, '2025-10-12'),

  -- Variante A (control) - segmento mobile
  (11,  7, 'A', 1,1,0,0,   0.0, '2025-10-01'),
  (12,  7, 'A', 1,1,1,1,   90.0, '2025-10-10'),
  (13,  8, 'A', 1,0,0,0,   0.0, '2025-10-03'),
  (14,  9, 'A', 1,1,1,0,   0.0, '2025-10-05'),
  (15, 10, 'A', 1,1,1,1,   60.0, '2025-10-07'),
  (16, 11, 'A', 1,1,0,0,   0.0, '2025-10-09'),

  -- Variante B (tratamiento) - segmento web
  (17,  1, 'B', 1,1,1,1, 140.0, '2025-10-15'),
  (18,  2, 'B', 1,1,1,1, 110.0, '2025-10-16'),
  (19,  3, 'B', 1,1,1,0,   0.0, '2025-10-18'),
  (20,  4, 'B', 1,1,1,1,  95.0, '2025-10-19'),
  (21, 13, 'B', 1,1,1,1, 105.0, '2025-10-20'),
  (22, 14, 'B', 1,1,0,0,   0.0, '2025-10-21'),
  (23, 17, 'B', 1,1,1,0,   0.0, '2025-10-22'),
  (24, 19, 'B', 1,0,0,0,   0.0, '2025-10-23'),

  -- Variante B (tratamiento) - segmento mobile
  (25,  7, 'B', 1,1,1,1,  85.0, '2025-10-15'),
  (26,  8, 'B', 1,1,1,0,   0.0, '2025-10-16'),
  (27,  9, 'B', 1,1,1,1,  70.0, '2025-10-17'),
  (28, 10, 'B', 1,1,1,1,  65.0, '2025-10-18'),
  (29, 11, 'B', 1,1,0,0,   0.0, '2025-10-19'),
  (30, 12, 'B', 1,1,1,1,  95.0, '2025-10-20'),
  (31, 15, 'B', 1,0,0,0,   0.0, '2025-10-21'),
  (32, 16, 'B', 1,1,1,0,   0.0, '2025-10-22'),
  (33, 18, 'B', 1,1,1,1,  75.0, '2025-10-23'),
  (34, 20, 'B', 1,1,1,1,  88.0, '2025-10-24');


## 2. Exploración inicial del dataset

Antes de simular escenarios, hagamos una exploración básica con SQL.

> **Tip:** estos ejercicios son ideales para hacer en modalidad **Coding Together** y luego dejar 1–2 preguntas como **Breakrooms**.


### Ejercicio 2.1 — Inspeccionar las primeras filas

**Fundamento teórico**

Siempre que recibimos un nuevo dataset, el primer paso es **mirar los datos**:

- Ver algunas filas completas.
- Confirmar tipos de columnas y rangos de valores.
- Detectar rápidamente valores raros (ej. `purchased=1` pero `viewed_product=0`).

**Descripción del ejercicio**

1. Lista las primeras 10 filas de `funnel_sessions`.  
2. Verifica visualmente que las combinaciones de flags (`viewed_product`, `added_to_cart`, etc.) tengan sentido.

**Pistas**

- Usa `SELECT * FROM funnel_sessions LIMIT 10;`.  
- Si quieres ordenar por fecha: añade `ORDER BY session_date`.


In [ ]:
-- Solución sugerida 2.1
SELECT *
FROM funnel_sessions
ORDER BY session_date, session_id
LIMIT 10;


### Ejercicio 2.2 — Conteo de sesiones por variante y segmento

**Fundamento teórico**

Antes de analizar conversiones, necesitamos saber **cuántas observaciones** tenemos por grupo:

- ¿Hay balance entre **A** y **B**?
- ¿Hay muchos más usuarios **web** que **mobile**?

Esto afecta la confiabilidad de cualquier comparación.

**Descripción del ejercicio**

Calcula el número de sesiones por combinación de:

- `variant` (A/B)  
- `segment` (web/mobile)

**Pistas**

- Necesitas unir `funnel_sessions` con `users` por `user_id`.  
- Luego agrupar por `variant` y `segment`.  
- Usa `COUNT(*)` para contar sesiones.


In [ ]:
-- Solución sugerida 2.2
SELECT
  fs.variant,
  u.segment,
  COUNT(*) AS n_sessions
FROM funnel_sessions fs
JOIN users u USING (user_id)
GROUP BY fs.variant, u.segment
ORDER BY fs.variant, u.segment;


## 3. A/B testing en funnels y simulación de mejoras

En esta sección vamos a:

1. Calcular las **tasas de conversión reales** por etapa y variante.  
2. Definir un escenario simulado donde la variante B mejora una etapa del funnel.  
3. Estimar el efecto en la conversión final a `purchase`.


### Ejercicio 3.1 — Tasas de conversión por variante (Actual)

**Fundamento teórico**

Un funnel simple se puede describir con **proporciones de usuarios** que alcanzan cada etapa:

- `viewed_product` → `added_to_cart`  
- `added_to_cart` → `begin_checkout`  
- `begin_checkout` → `purchased`  

Comparar estas tasas entre A y B nos dice **dónde** está la diferencia.

**Descripción del ejercicio**

Para cada variante (`A`, `B`), calcula:

1. Número de sesiones con `viewed_product = 1`.  
2. Número de sesiones con `added_to_cart = 1`.  
3. Número de sesiones con `begin_checkout = 1`.  
4. Número de sesiones con `purchased = 1`.  

Y luego calcula las tasas:

- `add_to_cart_rate    = added_to_cart / viewed_product`  
- `checkout_rate       = begin_checkout / added_to_cart`  
- `purchase_rate_final = purchased / viewed_product`

**Pistas**

- Usa expresiones como `SUM(CASE WHEN viewed_product = 1 THEN 1 ELSE 0 END)` para contar.  
- Puedes envolver las tasas con `ROUND(valor, 3)` para ver 3 decimales.


In [ ]:
-- Solución sugerida 3.1
WITH base AS (
  SELECT
    variant,
    SUM(CASE WHEN viewed_product = 1 THEN 1 ELSE 0 END) AS n_view,
    SUM(CASE WHEN added_to_cart  = 1 THEN 1 ELSE 0 END) AS n_cart,
    SUM(CASE WHEN begin_checkout = 1 THEN 1 ELSE 0 END) AS n_chk,
    SUM(CASE WHEN purchased      = 1 THEN 1 ELSE 0 END) AS n_buy
  FROM funnel_sessions
  GROUP BY variant
)
SELECT
  variant,
  n_view,
  n_cart,
  n_chk,
  n_buy,
  ROUND(1.0 * n_cart / n_view, 3) AS add_to_cart_rate,
  ROUND(1.0 * n_chk  / n_cart, 3) AS checkout_rate,
  ROUND(1.0 * n_buy  / n_view, 3) AS purchase_rate_final
FROM base;


### Ejercicio 3.2 — Simular una mejora en una etapa del funnel (Variante B)

**Fundamento teórico**

Para hacer **modelado de escenarios** no necesitamos modificar los datos originales: podemos trabajar con **agregados**.

Ejemplo de hipótesis:

> "Si mejoramos el paso `begin_checkout` para la variante B en +5 puntos porcentuales  
> (ej. de 0.60 a 0.65), ¿cuántas compras adicionales obtendríamos?"

Asumiremos que:

- El uplift afecta solo la etapa `added_to_cart → begin_checkout`.  
- Las tasas posteriores se mantienen constantes.

**Descripción del ejercicio**

1. Toma los agregados de la variante B del ejercicio anterior.  
2. Supón una nueva tasa `checkout_rate_sim = checkout_rate_actual + 0.05`.  
3. Calcula:
   - `n_chk_sim = n_cart * checkout_rate_sim`  
   - `n_buy_sim = n_chk_sim * (n_buy / n_chk)`  (mantienes la tasa `begin_checkout → purchase`)  
4. Compara:
   - `n_buy` (compras actuales) vs `n_buy_sim` (compras simuladas).  
   - `delta_buy = n_buy_sim - n_buy`.

**Pistas**

- Puedes reutilizar el CTE `base` del ejercicio 3.1 y filtrar `variant = 'B'`.  
- Haz los cálculos con `1.0 *` para obtener decimales.  
- Usa alias claros: `checkout_rate_actual`, `checkout_rate_sim`, etc.


In [ ]:
-- Solución sugerida 3.2
WITH base AS (
  SELECT
    variant,
    SUM(CASE WHEN viewed_product = 1 THEN 1 ELSE 0 END) AS n_view,
    SUM(CASE WHEN added_to_cart  = 1 THEN 1 ELSE 0 END) AS n_cart,
    SUM(CASE WHEN begin_checkout = 1 THEN 1 ELSE 0 END) AS n_chk,
    SUM(CASE WHEN purchased      = 1 THEN 1 ELSE 0 END) AS n_buy
  FROM funnel_sessions
  WHERE variant = 'B'
  GROUP BY variant
),
rates AS (
  SELECT
    variant,
    n_view, n_cart, n_chk, n_buy,
    1.0 * n_chk / n_cart AS checkout_rate_actual,
    1.0 * n_buy / n_chk  AS buy_given_chk_rate
  FROM base
)
SELECT
  variant,
  n_view, n_cart, n_chk, n_buy,
  ROUND(checkout_rate_actual, 3) AS checkout_rate_actual,
  ROUND(checkout_rate_actual + 0.05, 3) AS checkout_rate_sim,
  ROUND(n_cart * (checkout_rate_actual + 0.05), 2) AS n_chk_sim,
  ROUND(n_cart * (checkout_rate_actual + 0.05) * buy_given_chk_rate, 2) AS n_buy_sim,
  ROUND(
    n_cart * (checkout_rate_actual + 0.05) * buy_given_chk_rate - n_buy,
    2
  ) AS delta_buy
FROM rates;


## 4. Estimando impacto en métricas de negocio

Ahora queremos traducir el cambio en número de compras a **revenue**.

Supondremos que el `revenue` de nuestra tabla representa el valor real de cada compra.  
A partir de ahí podemos estimar:

- Ticket promedio actual (`avg_revenue_per_purchase`).  
- Revenue actual total.  
- Revenue simulado con `n_buy_sim` compras.


### Ejercicio 4.1 — Ticket promedio y revenue actual

**Fundamento teórico**

Si conocemos:

- Número de compras (`n_buy`).  
- Suma de revenue (`SUM(revenue)` para sesiones con `purchased = 1`).

Podemos estimar:

- `avg_revenue_per_purchase = SUM(revenue) / n_buy`.

**Descripción del ejercicio**

Para la variante B:

1. Calcula cuántas sesiones tienen `purchased = 1`.  
2. Calcula `SUM(revenue)` solo sobre esas sesiones.  
3. Obtén el ticket promedio por compra.  

**Pistas**

- Recuerda filtrar `variant = 'B'`.  
- Limita el cálculo de `SUM(revenue)` a `purchased = 1`.  
- Puedes usar otro CTE `stats_b` para reusar estos valores luego.


In [ ]:
-- Solución sugerida 4.1
WITH stats_b AS (
  SELECT
    COUNT(*) AS n_sessions,
    SUM(CASE WHEN purchased = 1 THEN 1 ELSE 0 END) AS n_buy,
    SUM(CASE WHEN purchased = 1 THEN revenue ELSE 0 END) AS revenue_total
  FROM funnel_sessions
  WHERE variant = 'B'
)
SELECT
  n_sessions,
  n_buy,
  revenue_total,
  ROUND(1.0 * revenue_total / n_buy, 2) AS avg_revenue_per_purchase
FROM stats_b;


### Ejercicio 4.2 — Revenue simulado con el nuevo funnel

**Fundamento teórico**

Si asumimos que el **ticket promedio** se mantiene constante, entonces:

- `revenue_sim = n_buy_sim * avg_revenue_per_purchase`.  
- `uplift_revenue = revenue_sim - revenue_actual`.

**Descripción del ejercicio**

1. Reutiliza el `n_buy_sim` del ejercicio 3.2 (puedes volver a calcularlo en un CTE).  
2. Reutiliza el `avg_revenue_per_purchase` del ejercicio 4.1.  
3. Calcula:
   - `revenue_actual` (solo sesiones B con `purchased = 1`).  
   - `revenue_sim`.  
   - `uplift_revenue`.

**Pistas**

- Puedes unir dos CTE: uno para **tasas y n_buy_sim**, otro para **revenue_actual y ticket promedio**.  
- Asegúrate de que ambos CTE tengan la misma columna `variant` para hacer un `JOIN`.


In [ ]:
-- Solución sugerida 4.2
WITH conv AS (
  -- Reutilizamos la lógica del ejercicio 3.2 para obtener n_buy_sim
  SELECT
    'B' AS variant,
    SUM(CASE WHEN viewed_product = 1 THEN 1 ELSE 0 END) AS n_view,
    SUM(CASE WHEN added_to_cart  = 1 THEN 1 ELSE 0 END) AS n_cart,
    SUM(CASE WHEN begin_checkout = 1 THEN 1 ELSE 0 END) AS n_chk,
    SUM(CASE WHEN purchased      = 1 THEN 1 ELSE 0 END) AS n_buy
  FROM funnel_sessions
  WHERE variant = 'B'
),
rates AS (
  SELECT
    variant,
    n_view, n_cart, n_chk, n_buy,
    1.0 * n_chk / n_cart AS checkout_rate_actual,
    1.0 * n_buy / n_chk  AS buy_given_chk_rate
  FROM conv
),
sim AS (
  SELECT
    variant,
    n_buy,
    -- checkout_rate_sim = checkout_rate_actual + 0.05
    n_cart * (checkout_rate_actual + 0.05) * buy_given_chk_rate AS n_buy_sim
  FROM rates
),
rev AS (
  SELECT
    'B' AS variant,
    SUM(CASE WHEN purchased = 1 THEN revenue ELSE 0 END) AS revenue_actual,
    SUM(CASE WHEN purchased = 1 THEN 1 ELSE 0 END) AS n_buy_actual
  FROM funnel_sessions
  WHERE variant = 'B'
)
SELECT
  sim.variant,
  sim.n_buy        AS n_buy_actual,
  ROUND(sim.n_buy_sim, 2) AS n_buy_sim,
  rev.revenue_actual,
  ROUND(1.0 * rev.revenue_actual / rev.n_buy_actual, 2) AS avg_rev_per_purchase,
  ROUND(sim.n_buy_sim * (1.0 * rev.revenue_actual / rev.n_buy_actual), 2) AS revenue_sim,
  ROUND(
    sim.n_buy_sim * (1.0 * rev.revenue_actual / rev.n_buy_actual) - rev.revenue_actual,
    2
  ) AS uplift_revenue
FROM sim
JOIN rev USING (variant);


## 5. Comparando escenarios por segmento

En la práctica, un equipo de producto querrá saber **qué segmento aporta más al uplift**:

- Quizá la mejora favorece más a usuarios mobile.  
- O los usuarios web tienen ticket promedio más alto.

Vamos a calcular métricas segmentadas.


### Ejercicio 5.1 — Revenue actual por segmento y variante

**Fundamento teórico**

Comparar métricas por segmento nos permite ver **heterogeneidad de efectos**:

- Un mismo cambio de UI puede ayudar más en pantallas pequeñas (mobile).  
- O los usuarios web pueden convertir menos pero con tickets más altos.

**Descripción del ejercicio**

Para cada combinación `(segment, variant)`:

1. Cuenta cuántas sesiones hay.  
2. Cuenta cuántas sesiones terminan en `purchased = 1`.  
3. Calcula el `revenue_total`.  
4. Calcula el `ticket_promedio` (`revenue_total / n_buy`, si `n_buy > 0`).

**Pistas**

- Necesitas `JOIN users u USING (user_id)`.  
- Agrupa por `segment` y `variant`.  
- Usa `CASE` para evitar dividir por cero (`NULLIF(n_buy, 0)`).


In [ ]:
-- Solución sugerida 5.1
SELECT
  u.segment,
  fs.variant,
  COUNT(*) AS n_sessions,
  SUM(CASE WHEN fs.purchased = 1 THEN 1 ELSE 0 END) AS n_buy,
  SUM(CASE WHEN fs.purchased = 1 THEN fs.revenue ELSE 0 END) AS revenue_total,
  ROUND(
    1.0 * SUM(CASE WHEN fs.purchased = 1 THEN fs.revenue ELSE 0 END)
    / NULLIF(SUM(CASE WHEN fs.purchased = 1 THEN 1 ELSE 0 END), 0),
    2
  ) AS avg_revenue_per_purchase
FROM funnel_sessions fs
JOIN users u USING (user_id)
GROUP BY u.segment, fs.variant
ORDER BY u.segment, fs.variant;


### Ejercicio 5.2 — ¿Qué segmento aporta más al uplift simulado?

**Fundamento teórico**

Aunque el uplift de conversión (en %) sea igual para todos, el impacto en revenue puede:

- Ser mayor donde hay más volumen de usuarios.  
- O donde el ticket promedio es más alto.

**Descripción del ejercicio**

1. Elige una hipótesis simple, por ejemplo:
   - "Aplicar la misma mejora de `+5 p.p.` en `checkout_rate` para todos los segmentos de la variante B".  
2. Para cada `segment`:
   - Estima `n_buy_sim_segment` usando la misma lógica del ejercicio 3.2 pero filtrando por segmento.  
   - Usa el ticket promedio de ese segmento para obtener `revenue_sim_segment`.  
3. Compara `uplift_revenue_segment = revenue_sim_segment - revenue_actual_segment`.  

**Pistas**

- Puedes empezar con un segmento a la vez (ej. `segment = 'mobile'`) para no complicarte.  
- Si no alcanzas a escribir toda la consulta en clase, al menos deja clara la **estrategia** de cálculo.


In [ ]:
-- Espacio para que intentes la solución 5.2 durante la clase
-- Sugerencia: copia la estructura de los CTE de 3.2 y 4.2 y añade el filtro por segmento.

-- Ejemplo de esqueleto:
-- WITH conv AS (
--   SELECT ...
--   FROM funnel_sessions fs
--   JOIN users u USING (user_id)
--   WHERE fs.variant = 'B'
--     AND u.segment = 'mobile'
-- ), rates AS (...), sim AS (...), rev AS (...)
-- SELECT ...
;


## 6. Visualizando funnels para stakeholders (Google Sheets)

Aunque estamos trabajando con SQLite, muchas veces es útil llevar los resultados a **Google Sheets** para:

- Construir gráficos rápidos.
- Compartir con personas no técnicas.

### Ejercicio 6.1 — Tabla de resumen Actual vs Simulado

**Fundamento teórico**

Una buena tabla de resumen para stakeholders debe ser:

- **Compacta:** pocas métricas clave.  
- **Comparativa:** columnas lado a lado (Actual vs Simulado).  
- **Interpretables:** títulos claros y unidades (conversiones, %, dinero).

**Descripción del ejercicio**

1. Toma los resultados numéricos de:
   - Ejercicio 3.1 (tasas por etapa, Actual).  
   - Ejercicio 3.2 y 4.2 (compras y revenue simulados).  
2. Construye una tabla manual en Google Sheets con columnas:

   | Métrica                          | Actual | Simulado |
   |----------------------------------|--------|----------|
   | Compras totales variante B       |        |          |
   | Tasa de purchase (view → buy) B  |        |          |
   | Revenue total variante B         |        |          |
   | Revenue por compra (ticket medio)|        |          |

3. Llena la tabla con los valores que calculaste en SQLite.

**Pistas**

- Copia y pega los resultados de SQLite a Sheets o escribe los números manualmente.  
- Asegúrate de usar siempre las mismas unidades (por ejemplo, pesos colombianos).


### Ejercicio 6.2 — Gráfico de columnas Actual vs Simulado

**Fundamento teórico**

Los gráficos de **columnas agrupadas** son muy útiles para comparar 2 escenarios:

- Cada grupo representa una métrica.  
- Cada color/columna: Actual vs Simulado.

**Descripción del ejercicio**

En Google Sheets:

1. Selecciona tu tabla de resumen del ejercicio 6.1.  
2. Inserta un **gráfico de columnas**.  
3. Ajusta:
   - Título del gráfico (ej. *Impacto simulado variante B*).  
   - Nombres de ejes si es necesario.  
4. Añade un pequeño texto debajo del gráfico (en la hoja o en una diapositiva) con 3–5 frases que expliquen:
   - Qué cambió en la conversión.  
   - Cuánto revenue adicional se estima.  
   - Cualquier suposición importante (ej. ticket promedio constante).

**Pistas**

- Si tus métricas tienen escalas muy distintas (ej. % y dinero), considera hacer **dos gráficos**.  
- Recuerda que este tipo de visualización suele terminar en una **presentación** o reporte ejecutivo.


## 7. Cierre y siguientes pasos

En este webinar:

- Practicaste cómo **modelar escenarios** de mejora en un funnel usando solo SQL.  
- Tradujiste cambios en tasas de conversión a **impacto económico** (revenue).  
- Viste cómo **segmentar** resultados por tipo de usuario (web vs mobile).  
- Preparaste una estructura para **visualizar y comunicar** resultados a stakeholders.

### 7.1 Qué deberías llevarte

1. Un entendimiento claro de que un funnel no es solo un gráfico bonito:
   - Es una herramienta para **cuantificar hipótesis** de producto.  
2. La idea de que **simular** mejoras antes de implementar cambios ayuda a priorizar:
   - No todas las etapas o segmentos tienen el mismo potencial de impacto.  
3. La conexión entre:
   - Métricas de producto (conversiones).  
   - Métricas de negocio (ingresos).

### 7.2 Siguientes pasos sugeridos

- Repetir el ejercicio con:
  - Otra etapa objetivo (ej. `add_to_cart_rate` en lugar de `checkout_rate`).  
  - Otro segmento (ej. solo `mobile`).  
- Probar distintos valores de uplift (+3 p.p., +10 p.p.) y ver cómo cambia el impacto.  
- Extender el análisis:
  - Añadir más atributos de usuario (plan, canal de adquisición).  
  - Crear más escenarios (ej. peor caso, caso conservador, caso optimista).  
- Preparar un breve **memo ejecutivo** (media página) explicando:
  - Qué cambio se propone.  
  - Qué impacto estimado tendría.  
  - En qué segmento se recomienda priorizar.

En módulos posteriores complementaremos este enfoque con:

- **Estadística formal de A/B testing** (p-values, intervalos de confianza).  
- Técnicas de **experimentación controlada** a mayor escala.
